In [1]:
data = read.csv("0914_categorized.csv")

library(dplyr)
library(xgboost)
library(DT)
library(Matrix)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice



In [2]:
data = data[,-1]

train = data[data$label != "testset", ] 
data4 = train  %>% filter(length == 4) %>% unique()
wide4 = data4  %>% group_by(acc_id) %>% mutate(index = row_number()) %>% as.data.frame()
wide4 = reshape(wide4, idvar = "acc_id", timevar = "index", direction = "wide")
id_4  = wide4 %>% dplyr::select(acc_id)
wide4 = wide4 %>% dplyr::select(-wk.1, -wk.2, -wk.3,-wk.4, -acc_id)
wide4$label.1 = as.integer(wide4$label.1) - 1
wide4$label.1[which(wide4$label.1 == 4)] = 3
ans4          = wide4$label.1
wide4         = wide4 %>% dplyr::select(-label.1, -label.2, -label.3, -label.4)
data_matrix4  = xgb.DMatrix(data  = as.matrix(wide4), 
                             label = ans4)   

In [3]:
f1_calcul = function(table_eval){
  {
    fffmea = 8 / {
      1 / (table_eval[1,1] / sum(table_eval[,1])) +
        1 / (table_eval[2,2] / sum(table_eval[,2])) +
        1 / (table_eval[3,3] / sum(table_eval[,3])) +
        1 / (table_eval[4,4] / sum(table_eval[,4])) +
        1 / (table_eval[1,1] / sum(table_eval[1,])) +
        1 / (table_eval[2,2] / sum(table_eval[2,])) +
        1 / (table_eval[3,3] / sum(table_eval[3,])) +
        1 / (table_eval[4,4] / sum(table_eval[4,]))
    } 
    return(fffmea)
  }}


In [4]:
as.numeric.factor = function(x) {as.numeric(levels(x))[x]}

f1.max = function(preds, dtrain) {
  labels = getinfo(dtrain, "label")
  preds = matrix(preds, ncol=4, byrow=TRUE)
  table_eval = table(apply(preds,1,which.max)-1, labels)
  err = f1_calcul(table_eval)
  return(list(metric = "f1", value = err))
}

In [ ]:
best_param = list()
best_seednumber = 1234
best_f1score = 0
best_f1score_index = 0
library(xgboost)
for (iter in 1:20) 
    
{
  xgb_params4 <- list(objective = "multi:softprob",
                eval_metric = f1.max,
                num_class = 4,
                max_depth = sample(40:60, 1),
                eta = runif(1, .01, .3),
                gamma = runif(1, 0.0, 0.2), 
                subsample = runif(1, .6, .9),
                colsample_bytree = runif(1, .5, .8), 
                min_child_weight = sample(1:40, 1),
                max_delta_step = sample(1:10, 1)
  )
  cv.nround = 1000
  cv.nfold = 5
  seed.number = sample.int(10000, 1)[[1]]
  set.seed(seed.number)
  mdcv <- xgb.cv(data=data_matrix4, label = ans4, params = xgb_params4, 
                 nfold=cv.nfold, nrounds=cv.nround, prediction = TRUE, 
                 maximize = TRUE, verbose = TRUE,
                 early_stopping_rounds=8,
                 print_every_n = 5,
                 callback(cb.cv.predict(save_models = TRUE)))
  
  max_f1score = max(mdcv$evaluation_log[, test_f1_mean])
  max_f1score_index = which.min(mdcv$evaluation_log[, test_f1_mean])
   
  if (max_f1score > best_f1score) {
   best_f1score = max_f1score
   best_f1score_index = max_f1score_index
   best_seednumber = seed.number
   best_param = xgb_params4
  }
}


Warning message in xgb.get.DMatrix(data, label, missing):
“xgboost: label will be ignored.”

[1]	train-f1:0.553654+0.020183	test-f1:0.509465+0.011722 
Multiple eval metrics are present. Will use test_f1 for early stopping.
Will train until test_f1 hasn't improved in 8 rounds.

[6]	train-f1:0.609065+0.003724	test-f1:0.562619+0.018138 
[11]	train-f1:0.626710+0.003928	test-f1:0.570269+0.012802 
[16]	train-f1:0.644902+0.004388	test-f1:0.581875+0.009943 
[21]	train-f1:0.656592+0.006428	test-f1:0.587644+0.009963 
[26]	train-f1:0.669738+0.003649	test-f1:0.591525+0.015668 
[31]	train-f1:0.684474+0.003661	test-f1:0.595522+0.008239 
[36]	train-f1:0.697788+0.005576	test-f1:0.594539+0.008840 
Stopping. Best iteration:
[31]	train-f1:0.684474+0.003661	test-f1:0.595522+0.008239



Warning message in xgb.get.DMatrix(data, label, missing):
“xgboost: label will be ignored.”